In [22]:
import cv2
import numpy as np
from deepface import DeepFace
import faiss
import os

# Load YuNet model for face detection
yunet = cv2.FaceDetectorYN.create(
    model="face_detection_yunet_2023mar.onnx",  # Pre-trained ONNX model path
    config="",
    input_size=(320, 320),  # Input image size
    score_threshold=0.9,
    nms_threshold=0.3,
    top_k=5000
)

# Load FaceNet model for embeddings
facenet_model = "Facenet"  # You can also use 'Facenet512' for higher accuracy


In [23]:
def detect_and_crop_faces(image_path, return_boxes=False):
    """
    Detect faces in an image, crop them, and optionally return face bounding boxes.

    Args:
        image_path (str): Path to the input image.
        return_boxes (bool): Whether to return face bounding boxes.

    Returns:
        list: List of cropped face images.
        list (optional): List of bounding boxes [(x, y, width, height), ...].
    """
    # Read the input image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read {image_path}")
        return [] if not return_boxes else ([], [])

    # Set YuNet input size
    height, width = img.shape[:2]
    yunet.setInputSize((width, height))
    
    # Detect faces
    _, faces = yunet.detect(img)

    cropped_faces = []
    face_boxes = []

    if faces is not None:
        for face in faces:
            x, y, w, h = face[:4].astype(int)
            # Crop the face from the image
            cropped_face = img[y:y+h, x:x+w]
            cropped_faces.append(cropped_face)
            face_boxes.append((x, y, w, h))  # Store the bounding box coordinates

    if return_boxes:
        return cropped_faces, face_boxes
    return cropped_faces


In [24]:
# Step 1: Load the pre-trained FaceNet model


def get_embeddings(face_images):
    embeddings = []
    for face_img in face_images:
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        embedding = DeepFace.represent(img_path=face_rgb, model_name=facenet_model,enforce_detection=False)[0]['embedding']
        embeddings.append(np.array(embedding, dtype=np.float32))
    return embeddings


In [25]:
# Initialize FAISS index
embedding_dimension = 128  # FaceNet output dimension
index = faiss.IndexFlatL2(embedding_dimension)  # L2 (Euclidean) distance
photo_ids = []  # Keep track of photo IDs to map results

def process_and_store_images(image_folder):
    for photo_id, image_file in enumerate(os.listdir(image_folder)):
        image_path = os.path.join(image_folder, image_file)
        faces = detect_and_crop_faces(image_path)
        if faces:
            embeddings = get_embeddings(faces)
            for embedding in embeddings:
                index.add(np.expand_dims(embedding, axis=0))  # Add embedding to FAISS
                photo_ids.append(image_file)
    print(f"Stored {len(photo_ids)} faces in FAISS index.")


In [26]:
def search_similar_faces(query_image, top_k=5):
    faces = detect_and_crop_faces(query_image)
    if not faces:
        print("No face detected in query image.")
        return

    query_embeddings = get_embeddings(faces)

    for query_embedding in query_embeddings:
        distances, indices = index.search(np.expand_dims(query_embedding, axis=0), top_k)
        print("Top Matches:")
        for dist, idx in zip(distances[0], indices[0]):
            if idx >= 0:
                print(f"Image: {photo_ids[idx]}, Distance: {dist}")


In [27]:
image_folder = "drive"  # Folder with all images
process_and_store_images(image_folder)

Stored 64 faces in FAISS index.


In [28]:
query_image = "img_3.png"
search_similar_faces(query_image, top_k=5)

Top Matches:
Image: 1F2A5973.JPG, Distance: 12.011140823364258
Image: _SAN4237.JPG, Distance: 56.65167999267578
Image: _SAN4171.JPG, Distance: 92.5196762084961
Image: 1F2A5501.JPG, Distance: 103.70606231689453
Image: _SAN4171.JPG, Distance: 105.64873504638672


In [29]:
faiss.write_index(index, "faiss.index")